In [31]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor

import keys
import numpy as np
import pandas as pd

In [88]:
class TwitterClient():
    def __init__(self,twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)
        self.twitter_user = twitter_user
    def get_twitter_client_api(self):
        return self.twitter_client
    def get_user_timeline_tweets(self,num):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline,id=self.twitter_user).items(num):
            tweets.append(tweet)
        return tweets
    def get_user_friend_list(self,num):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends,id=self.twitter_user).items(num):
            friend_list.append(friend)
        return friend_list
    # More calls on here http://docs.tweepy.org/en/v3.5.0/api.html


class TwitterAuthenticator():
    def authenticate_twitter_app(self):
        auth = OAuthHandler(keys.consumer_api_key, keys.consumer_api_secret_key)
        auth.set_access_token(keys.access_token,keys.access_token_secret)
        return auth
        

class TwitterStreamer():
    
    def __init__(self):
        self.twitter_authenticator = TwitterAuthenticator()
        
    def stream_tweets(self,fetched_tweets_filename, hash_tag_list):
        listener = TwitterListener()
        auth = self.twitter_authenticator.authenticate_twitter_app()

        stream = Stream(auth,listener)

        stream.filter(track=hash_tag_list)


class TwitterListener(StreamListener):
    def __init__(self):
        self.fetched_tweets_filename = fetched_tweets_filename
    def on_data(self,data):
        try:
            with open(self.fetched_tweets_filename,'a') as tf:
                tf.write(data)
            print(data)
        except BaseException as e:
            print('Error:',str(e))
        return True
    def on_error(self,status):
        if status == 420:
            return False  # Rate Limit Error
        print(status)
        
class TwitterAnalyzer():
    def tweets_to_DataFrame(self,tweets):
        df = pd.DataFrame()
        for col in tweets[0]._json.keys():
            df[col] = np.array([tweet._json[col] for tweet in tweets])
        
        return df

In [89]:
if __name__ == '__main__':
#     hash_tag_list = ['Blizzard','hearthstone','warcraft','overwatch','starcraft']
#     fetched_tweets_filename = 'tweets.json'
#     twitter_streamer = TwitterStreamer()
#     twitter_streamer.stream_tweets(fetched_tweets_filename,hash_tag_list)
#      print(dir(TwitterClient('lllllllll520').get_user_timeline_tweets(2)[0]))
    tweets = TwitterClient().get_twitter_client_api().user_timeline(screen_name='SoLegitHS',count='20')
    df = TwitterAnalyzer().tweets_to_DataFrame(tweets)

In [90]:
df

,created_at,id,id_str,text,truncated,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang
0,Wed Jun 26 13:26:53 +0000 2019,1143873267008098304,1143873267008098304,@HotMEOWTH @solegit Huge fan of @solegit,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/android"" ...",1143858715310739466,1143858715310739466,702987004917624832,...,None,None,None,None,False,0,6,False,False,en
1,Sun Jun 23 02:39:03 +0000 2019,1142623069896237056,1142623069896237056,@coL_Asmodai AAECAf0ECsUEkAe4CO72AsP4Asb4AqCAA...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://mobile.twitter.com"" rel=""nofo...",1142613560121413632,1142613560121413632,3168980351,...,None,None,None,None,False,0,5,False,False,in
2,Sun Jun 23 02:01:16 +0000 2019,1142613560121413632,1142613560121413632,Was messing around with wacky mage stuff on EU...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://mobile.twitter.com"" rel=""nofo...",None,None,None,...,None,None,None,None,False,13,78,False,False,en
3,Sat Jun 22 04:22:40 +0000 2019,1142286757569843200,1142286757569843200,https://t.co/6SuJEYZ5Ju\n\nplaying super secre...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://mobile.twitter.com"" rel=""nofo...",None,None,None,...,None,None,None,None,False,0,2,False,False,en
4,Wed Jun 19 04:08:48 +0000 2019,1141196105172078593,1141196105172078593,https://t.co/6SuJEYHuRW\n\ntop 15 NA undumpste...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://mobile.twitter.com"" rel=""nofo...",None,None,None,...,None,None,None,None,False,1,2,False,False,en
5,Mon Jun 17 01:36:29 +0000 2019,1140432997121810433,1140432997121810433,@GO_Gallon @Liquid_hsdog @GamersOrigin You're ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/android"" ...",1140416712057036800,1140416712057036800,710679348148219904,...,None,None,None,None,False,0,3,False,False,en
6,Mon Jun 17 01:13:24 +0000 2019,1140427189956911107,1140427189956911107,@Liquid_hsdog @itshafu Congrats! You're the best,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/android"" ...",1140422067201855489,1140422067201855489,2575478810,...,None,None,None,None,False,0,2,False,False,en
7,Mon Jun 17 00:43:03 +0000 2019,1140419548866215936,1140419548866215936,@kensanityHS Dog won Vegas with freeze mage :),False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/android"" ...",1140419316258443264,1140419316258443264,701511365253419008,...,None,None,None,None,False,0,1,False,False,en
8,Mon Jun 17 00:36:21 +0000 2019,1140417865775906816,1140417865775906816,@Ostkaka @Battlefy,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/android"" ...",1140414730357620737,1140414730357620737,2349120332,...,None,None,None,None,False,0,9,False,False,und
9,Mon Jun 17 00:29:49 +0000 2019,1140416221487099904,1140416221487099904,LOOOOOOOOOOOOOOOOOOOL,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/android"" ...",None,None,None,...,None,None,None,None,False,0,13,False,False,und


TypeError: 'Status' object is not iterable